### 本次作业先用老师的样本数据完成 完整的大数据后续再处理提交比赛结果 先调整模型

# 读取数据

In [1]:
import pandas as pd

# 加载小样本
user_log = pd.read_csv('sample_user_log.csv', dtype={'time_stamp':'str'})
user_log#用户信息
#user_id表示购买者ID  item_id表示商品的ID cat_id表示商品所属类别的ID seller_id表示卖家的ID 
#brand_id表示商品的品牌ID time_stamp表示购买的时间戳   action_type表示用户行为 0单击 1添加购物车 2购买 3添加到收藏夹

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type
0,89355,664924,1429,3136,579.0,1111,2
1,89355,131438,1505,780,516.0,1110,0
2,89355,673082,1429,3136,579.0,1110,0
3,89355,664924,1429,3136,579.0,1110,0
4,89355,183665,1505,780,516.0,1110,0
...,...,...,...,...,...,...,...
2473900,263674,176091,389,4143,4594.0,1111,0
2473901,263674,794804,662,4143,4594.0,1111,0
2473902,263674,794804,662,4143,4594.0,1111,0
2473903,263674,176091,389,4143,4594.0,1111,0


In [2]:
user_info = pd.read_csv('sample_user_info.csv')
user_info#用户信息表  user_id表示购买者ID 
#age_range表示年龄范围[0,18）为1  [18,24]为2 [25,29]为3 [30,34]为4 [35,39]为5 [40,49]为6 [50+]为7.8  0和null表示未知 
#gender表示性别 0女  1男

,user_id,age_range,gender
0,191719,4.0,0.0
1,391524,5.0,1.0
2,184971,4.0,0.0
3,396296,4.0,0.0
4,39031,4.0,0.0
...,...,...,...
19107,152607,4.0,0.0
19108,37882,4.0,1.0
19109,160530,2.0,0.0
19110,277791,3.0,NaN


In [3]:
train_data1 = pd.read_csv('train.csv')
train_data1#user_id表示购买者的ID  merchant_id表示卖家的ID  这个跟上面的seller应该是同一个吧？
#label是标签 1代表该用户是该商家的重复买家   0代表不是  -1表示该卖家不是该商家的新用户 不在我们预测范围内 

,user_id,merchant_id,label
0,365952,1203,0
1,42624,946,0
2,240000,2278,0
3,177792,951,0
4,322944,1892,0
...,...,...,...
17832,411263,3826,0
17833,415103,3444,0
17834,90239,4173,0
17835,355199,993,0


In [4]:
submission = pd.read_csv('test.csv')
submission#  prob  给定客户是给定商家的重复购买者的概率，取值在[0, 1]

,user_id,merchant_id,prob
0,40320,4173,0
1,309120,4775,1
2,117120,3826,0
3,127104,4048,1
4,331392,1978,0
...,...,...,...
6051,47231,1748,0
6052,59519,798,0
6053,263039,639,0
6054,263039,3954,0


In [5]:
train_data = pd.read_csv('train_format2.csv')
train_data#user_id表示购买者ID  
#age_range表示年龄范围[0,18）为1  [18,24]为2 [25,29]为3 [30,34]为4 [35,39]为5 [40,49]为6 [50+]为7.8  0和null表示未知 
#gender表示性别 0女 1男                merchant_id表示商家的ID 
#label是标签 1代表该用户是该商家的重复买家   0代表不是  -1表示该卖家不是该商家的新用户 不在我们预测范围内 
#{userid, merchantid}之间的每组交易中都记录有itemid, categoryid, brand_id, time，用#分隔。记录不按任何特定顺序排序。

,user_id,age_range,gender,merchant_id,label,activity_log
0,34176,6.0,0.0,944,-1,408895:1505:7370:1107:0
1,34176,6.0,0.0,412,-1,17235:1604:4396:0818:0#954723:1604:4396:0818:0...
2,34176,6.0,0.0,1945,-1,231901:662:2758:0818:0#231901:662:2758:0818:0#...
3,34176,6.0,0.0,4752,-1,174142:821:6938:1027:0
4,34176,6.0,0.0,643,-1,716371:1505:968:1024:3
...,...,...,...,...,...,...
7030718,229247,4.0,2.0,2000,-1,610483:737:3894:0516:2#610483:737:3894:0524:0
7030719,229247,4.0,2.0,579,-1,179514:420:3610:1014:0#1007071:420:3610:1014:0...
7030720,229247,4.0,2.0,1860,-1,980351:1130:918:1016:3#153613:1130:918:1016:0#...
7030721,229247,4.0,2.0,4140,0,308321:737:5697:1111:0#308321:737:5697:1111:0#...


# 数据预处理

In [6]:
# 数据有点眼花缭乱  需要整理一下
train_data1['origin'] = 'train'#train_data1这一列添加了一个字段 并且内容全是train
train_data1

,user_id,merchant_id,label,origin
0,365952,1203,0,train
1,42624,946,0,train
2,240000,2278,0,train
3,177792,951,0,train
4,322944,1892,0,train
...,...,...,...,...
17832,411263,3826,0,train
17833,415103,3444,0,train
17834,90239,4173,0,train
17835,355199,993,0,train


In [7]:
submission['origin'] = 'test'#submission这一列添加了一个字段 并且内容全是train
submission

,user_id,merchant_id,prob,origin
0,40320,4173,0,test
1,309120,4775,1,test
2,117120,3826,0,test
3,127104,4048,1,test
4,331392,1978,0,test
...,...,...,...,...
6051,47231,1748,0,test
6052,59519,798,0,test
6053,263039,639,0,test
6054,263039,3954,0,test


### matrix数据预处理

In [8]:
matrix = pd.concat([train_data1, submission], ignore_index=True, sort=False)
matrix#这样train_data1和submission就合并了 

,user_id,merchant_id,label,origin,prob
0,365952,1203,0.0,train,NaN
1,42624,946,0.0,train,NaN
2,240000,2278,0.0,train,NaN
3,177792,951,0.0,train,NaN
4,322944,1892,0.0,train,NaN
...,...,...,...,...,...
23888,47231,1748,NaN,test,0.0
23889,59519,798,NaN,test,0.0
23890,263039,639,NaN,test,0.0
23891,263039,3954,NaN,test,0.0


In [9]:
matrix.drop(['prob'], axis=1, inplace=True)#丢掉prob这个字段
matrix

,user_id,merchant_id,label,origin
0,365952,1203,0.0,train
1,42624,946,0.0,train
2,240000,2278,0.0,train
3,177792,951,0.0,train
4,322944,1892,0.0,train
...,...,...,...,...
23888,47231,1748,NaN,test
23889,59519,798,NaN,test
23890,263039,639,NaN,test
23891,263039,3954,NaN,test


In [10]:
# 连接 user_info 表，通过user_id关联
matrix = matrix.merge(user_info, on='user_id', how='left')
matrix#这样就把上文的user_info这个表给连接起来了 这里增加了age_range和gender两列

,user_id,merchant_id,label,origin,age_range,gender
0,365952,1203,0.0,train,0.0,1.0
1,42624,946,0.0,train,2.0,0.0
2,240000,2278,0.0,train,3.0,0.0
3,177792,951,0.0,train,0.0,1.0
4,322944,1892,0.0,train,7.0,0.0
...,...,...,...,...,...,...
23888,47231,1748,NaN,test,0.0,0.0
23889,59519,798,NaN,test,3.0,0.0
23890,263039,639,NaN,test,2.0,1.0
23891,263039,3954,NaN,test,2.0,1.0


In [11]:
#看看有多少缺失值
matrix.isna().sum()#缺失值还不少

user_id           0
merchant_id       0
label          6056
origin            0
age_range       125
gender          366
dtype: int64

In [12]:
#age_range表示年龄范围[0,18）为1  [18,24]为2 [25,29]为3 [30,34]为4 [35,39]为5 [40,49]为6 [50+]为7.8  0和null表示未知 
matrix['age_range'].fillna(0, inplace=True)#对于age_range的空值用0来填充
# 0:female, 1:male, 2:unknown
matrix['gender'].fillna(2, inplace=True)#gender的控制用2来填充
#下面4句代码都是数据类型的转换
matrix['age_range'] = matrix['age_range'].astype('int8')
matrix['gender'] = matrix['gender'].astype('int8')
matrix['label'] = matrix['label'].astype('str')
matrix['user_id'] = matrix['user_id'].astype('int32')
matrix['merchant_id'] = matrix['merchant_id'].astype('int32')

In [13]:
matrix.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23893 entries, 0 to 23892
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      23893 non-null  int32 
 1   merchant_id  23893 non-null  int32 
 2   label        23893 non-null  object
 3   origin       23893 non-null  object
 4   age_range    23893 non-null  int8  
 5   gender       23893 non-null  int8  
dtypes: int32(2), int8(2), object(2)
memory usage: 793.3+ KB


In [14]:
#看看是否还有缺失值 #缺失值已经处理完毕了
matrix.isna().sum()

user_id        0
merchant_id    0
label          0
origin         0
age_range      0
gender         0
dtype: int64

### user_log的数据处理

In [15]:
# 使用merchant_id（原列名seller_id）  user_log表格中的seller_id其实就是merchant_id  他大舅他二舅都是他舅  把马甲换回来把
user_log.rename(columns={'seller_id':'merchant_id'}, inplace=True)
user_log#改名成功 

,user_id,item_id,cat_id,merchant_id,brand_id,time_stamp,action_type
0,89355,664924,1429,3136,579.0,1111,2
1,89355,131438,1505,780,516.0,1110,0
2,89355,673082,1429,3136,579.0,1110,0
3,89355,664924,1429,3136,579.0,1110,0
4,89355,183665,1505,780,516.0,1110,0
...,...,...,...,...,...,...,...
2473900,263674,176091,389,4143,4594.0,1111,0
2473901,263674,794804,662,4143,4594.0,1111,0
2473902,263674,794804,662,4143,4594.0,1111,0
2473903,263674,176091,389,4143,4594.0,1111,0


In [16]:
user_log.info()#可以看到数据类型

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2473905 entries, 0 to 2473904
Data columns (total 7 columns):
 #   Column       Dtype  
---  ------       -----  
 0   user_id      int64  
 1   item_id      int64  
 2   cat_id       int64  
 3   merchant_id  int64  
 4   brand_id     float64
 5   time_stamp   object 
 6   action_type  int64  
dtypes: float64(1), int64(5), object(1)
memory usage: 132.1+ MB


In [17]:
# 格式化
user_log['user_id'] = user_log['user_id'].astype('int32')
user_log['merchant_id'] = user_log['merchant_id'].astype('int32')
user_log['item_id'] = user_log['item_id'].astype('int32')
user_log['cat_id'] = user_log['cat_id'].astype('int32')
user_log.info()#上述的4个ID都换成了 int32类型的了 这样可能更加节省内存一些

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2473905 entries, 0 to 2473904
Data columns (total 7 columns):
 #   Column       Dtype  
---  ------       -----  
 0   user_id      int32  
 1   item_id      int32  
 2   cat_id       int32  
 3   merchant_id  int32  
 4   brand_id     float64
 5   time_stamp   object 
 6   action_type  int64  
dtypes: float64(1), int32(4), int64(1), object(1)
memory usage: 94.4+ MB


In [18]:
#看看是否有缺失值
user_log.isna().sum()#brand_id竟然有4149个缺失值

user_id           0
item_id           0
cat_id            0
merchant_id       0
brand_id       4149
time_stamp        0
action_type       0
dtype: int64

In [19]:
#看一下它的数据分布 然后决定用什么方案填充比较好  brand_id的物理意思是商品的品牌ID 
user_log['brand_id'].value_counts()#看到这个分布 我觉得用众数填充比较好  老师用的0填充  可是这里面没有0啊

1360.0    35345
3738.0    34662
1446.0    34490
5376.0    23194
1214.0    22634
          ...  
5928.0        1
5925.0        1
1482.0        1
3719.0        1
2665.0        1
Name: brand_id, Length: 6817, dtype: int64

In [20]:
#从这个的数据分布来看 最好做众数的填充
print(user_log['brand_id'].mode(),type(user_log['brand_id'].mode()))#注意这里用众数填充 mode()之后是个serise 
user_log['brand_id'].fillna(user_log['brand_id'].mode()[0],inplace=True)#要mode()[0]才能拿到具体的数字 否则这里填充不了
user_log['brand_id'].isnull().sum()#user_log 的 brand_id 这一列已经没有缺失值了  

0    1360.0
dtype: float64 <class 'pandas.core.series.Series'>


0

In [21]:
user_log['brand_id'] = user_log['brand_id'].astype('int32')#数据类型转换一下
user_log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2473905 entries, 0 to 2473904
Data columns (total 7 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   user_id      int32 
 1   item_id      int32 
 2   cat_id       int32 
 3   merchant_id  int32 
 4   brand_id     int32 
 5   time_stamp   object
 6   action_type  int64 
dtypes: int32(5), int64(1), object(1)
memory usage: 84.9+ MB


In [22]:
user_log

,user_id,item_id,cat_id,merchant_id,brand_id,time_stamp,action_type
0,89355,664924,1429,3136,579,1111,2
1,89355,131438,1505,780,516,1110,0
2,89355,673082,1429,3136,579,1110,0
3,89355,664924,1429,3136,579,1110,0
4,89355,183665,1505,780,516,1110,0
...,...,...,...,...,...,...,...
2473900,263674,176091,389,4143,4594,1111,0
2473901,263674,794804,662,4143,4594,1111,0
2473902,263674,794804,662,4143,4594,1111,0
2473903,263674,176091,389,4143,4594,1111,0


In [23]:
#准备把time_stamp这一列的数据转化为时间戳
user_log['time_stamp'] = pd.to_datetime(user_log['time_stamp'], format='%H%M')#H代表小时 M代表
user_log

,user_id,item_id,cat_id,merchant_id,brand_id,time_stamp,action_type
0,89355,664924,1429,3136,579,1900-01-01 11:11:00,2
1,89355,131438,1505,780,516,1900-01-01 11:10:00,0
2,89355,673082,1429,3136,579,1900-01-01 11:10:00,0
3,89355,664924,1429,3136,579,1900-01-01 11:10:00,0
4,89355,183665,1505,780,516,1900-01-01 11:10:00,0
...,...,...,...,...,...,...,...
2473900,263674,176091,389,4143,4594,1900-01-01 11:11:00,0
2473901,263674,794804,662,4143,4594,1900-01-01 11:11:00,0
2473902,263674,794804,662,4143,4594,1900-01-01 11:11:00,0
2473903,263674,176091,389,4143,4594,1900-01-01 11:11:00,0


### User特征处理

In [24]:
groups = user_log.groupby(['user_id'])#以user_id分组聚合
groups

In [25]:
# 用户交互行为数量 u1
temp = groups.size().reset_index().rename(columns={0:'u1'})
temp

,user_id,u1
0,9,87
1,21,620
2,22,471
3,45,20
4,48,47
...,...,...
19107,424069,100
19108,424074,27
19109,424076,47
19110,424110,51


In [26]:
matrix = matrix.merge(temp, on='user_id', how='left')#跟之前的matrix合并
matrix

,user_id,merchant_id,label,origin,age_range,gender,u1
0,365952,1203,0.0,train,0,1,46
1,42624,946,0.0,train,2,0,365
2,240000,2278,0.0,train,3,0,47
3,177792,951,0.0,train,0,1,234
4,322944,1892,0.0,train,7,0,186
...,...,...,...,...,...,...,...
23888,47231,1748,nan,test,0,0,128
23889,59519,798,nan,test,3,0,1286
23890,263039,639,nan,test,2,1,9
23891,263039,3954,nan,test,2,1,9


In [27]:
# 对于每个user_id 不重复的item_id的数量 => u2
temp = groups['item_id'].agg([('u2', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
# 对于每个user_id 不重复的cat_id的数量 => u3
temp = groups['cat_id'].agg([('u3', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['merchant_id'].agg([('u4', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['brand_id'].agg([('u5', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
matrix

,user_id,merchant_id,label,origin,age_range,gender,u1,u2,u3,u4,u5
0,365952,1203,0.0,train,0,1,46,29,12,16,16
1,42624,946,0.0,train,2,0,365,198,46,46,45
2,240000,2278,0.0,train,3,0,47,31,14,15,17
3,177792,951,0.0,train,0,1,234,105,23,35,36
4,322944,1892,0.0,train,7,0,186,106,34,40,39
...,...,...,...,...,...,...,...,...,...,...,...
23888,47231,1748,nan,test,0,0,128,97,28,39,40
23889,59519,798,nan,test,3,0,1286,540,55,93,96
23890,263039,639,nan,test,2,1,9,8,7,7,7
23891,263039,3954,nan,test,2,1,9,8,7,7,7


In [28]:
# 时间间隔特征 u6 按照小时
# 对于每个user_id 计算time_stamp的最小时间 => F_time, 最大时间max => L_time
temp = groups['time_stamp'].agg([('F_time', 'min'), ('L_time', 'max')]).reset_index()
temp['u6'] = (temp['L_time'] - temp['F_time']).dt.seconds/3600
matrix = matrix.merge(temp[['user_id', 'u6']], on='user_id', how='left')
# 统计操作类型为0，1，2，3的个数
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'u7', 1:'u8', 2:'u9', 3:'u10'})
matrix = matrix.merge(temp, on='user_id', how='left')
matrix

,user_id,merchant_id,label,origin,age_range,gender,u1,u2,u3,u4,u5,u6,u7,u8,u9,u10
0,365952,1203,0.0,train,0,1,46,29,12,16,16,4.933333,45.0,NaN,1.0,NaN
1,42624,946,0.0,train,2,0,365,198,46,46,45,5.866667,313.0,NaN,21.0,31.0
2,240000,2278,0.0,train,3,0,47,31,14,15,17,5.833333,42.0,NaN,5.0,NaN
3,177792,951,0.0,train,0,1,234,105,23,35,36,5.833333,177.0,NaN,11.0,46.0
4,322944,1892,0.0,train,7,0,186,106,34,40,39,5.866667,147.0,NaN,25.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23888,47231,1748,nan,test,0,0,128,97,28,39,40,5.816667,122.0,NaN,6.0,NaN
23889,59519,798,nan,test,3,0,1286,540,55,93,96,6.000000,1182.0,NaN,16.0,88.0
23890,263039,639,nan,test,2,1,9,8,7,7,7,5.783333,7.0,NaN,2.0,NaN
23891,263039,3954,nan,test,2,1,9,8,7,7,7,5.783333,7.0,NaN,2.0,NaN


In [29]:
# 商家特征处理
groups = user_log.groupby(['merchant_id'])
# 商家被交互行为数量 m1
temp = groups.size().reset_index().rename(columns={0:'m1'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
# 统计商家被交互的user_id, item_id, cat_id, brand_id 唯一值
temp = groups['user_id', 'item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={'user_id':'m2', 'item_id':'m3', 'cat_id':'m4', 'brand_id':'m5'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
# 统计商家被交互的action_type 唯一值
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'m6', 1:'m7', 2:'m8', 3:'m9'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
# 按照merchant_id 统计随机负采样的个数
temp = train_data[train_data['label']==-1].groupby(['merchant_id']).size().reset_index().rename(columns={0:'m10'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
matrix

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  import sys


,user_id,merchant_id,label,origin,age_range,gender,u1,u2,u3,u4,...,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10
0,365952,1203,0.0,train,0,1,46,29,12,16,...,1915,408,175,19,2,1639.0,4.0,201.0,71.0,3518
1,42624,946,0.0,train,2,0,365,198,46,46,...,1965,292,320,6,3,1809.0,6.0,104.0,46.0,2816
2,240000,2278,0.0,train,3,0,47,31,14,15,...,1125,254,72,12,2,928.0,5.0,138.0,54.0,2604
3,177792,951,0.0,train,0,1,234,105,23,35,...,574,210,117,8,2,483.0,1.0,63.0,27.0,1932
4,322944,1892,0.0,train,7,0,186,106,34,40,...,6852,664,554,69,3,6028.0,27.0,473.0,324.0,5471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23888,47231,1748,nan,test,0,0,128,97,28,39,...,72,33,14,2,1,66.0,NaN,2.0,4.0,356
23889,59519,798,nan,test,3,0,1286,540,55,93,...,4892,907,109,20,2,4323.0,12.0,304.0,253.0,8650
23890,263039,639,nan,test,2,1,9,8,7,7,...,480,201,9,2,2,418.0,1.0,49.0,12.0,1943
23891,263039,3954,nan,test,2,1,9,8,7,7,...,515,171,31,7,3,434.0,3.0,68.0,10.0,1592


In [30]:
# 按照user_id, merchant_id分组
groups = user_log.groupby(['user_id', 'merchant_id'])
temp = groups.size().reset_index().rename(columns={0:'um1'}) #统计行为个数
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={'item_id':'um2', 'cat_id':'um3', 'brand_id':'um4'}) #统计item_id, cat_id, brand_id唯一个数
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'um5', 1:'um6', 2:'um7', 3:'um8'})#统计不同action_type唯一个数
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['time_stamp'].agg([('first', 'min'), ('last', 'max')]).reset_index()
temp['um9'] = (temp['last'] - temp['first']).dt.seconds/3600
temp.drop(['first', 'last'], axis=1, inplace=True)
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left') #统计时间间隔
matrix

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """


,user_id,merchant_id,label,origin,age_range,gender,u1,u2,u3,u4,...,m10,um1,um2,um3,um4,um5,um6,um7,um8,um9
0,365952,1203,0.0,train,0,1,46,29,12,16,...,3518,8,4,2,1,7.0,NaN,1.0,NaN,0.016667
1,42624,946,0.0,train,2,0,365,198,46,46,...,2816,42,14,2,1,34.0,NaN,7.0,1.0,0.016667
2,240000,2278,0.0,train,3,0,47,31,14,15,...,2604,2,1,1,1,1.0,NaN,1.0,NaN,0.000000
3,177792,951,0.0,train,0,1,234,105,23,35,...,1932,11,4,1,1,10.0,NaN,1.0,NaN,0.016667
4,322944,1892,0.0,train,7,0,186,106,34,40,...,5471,19,10,6,1,10.0,NaN,9.0,NaN,0.016667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23888,47231,1748,nan,test,0,0,128,97,28,39,...,356,9,1,1,1,8.0,NaN,1.0,NaN,0.066667
23889,59519,798,nan,test,3,0,1286,540,55,93,...,8650,42,15,11,1,35.0,NaN,4.0,3.0,2.783333
23890,263039,639,nan,test,2,1,9,8,7,7,...,1943,2,1,1,1,1.0,NaN,1.0,NaN,0.000000
23891,263039,3954,nan,test,2,1,9,8,7,7,...,1592,1,1,1,1,NaN,NaN,1.0,NaN,0.000000


In [31]:
#用户购买点击比
matrix['r1'] = matrix['u9']/matrix['u7'] 
#商家购买点击比
matrix['r2'] = matrix['m8']/matrix['m6'] 
#不同用户不同商家购买点击比
matrix['r3'] = matrix['um7']/matrix['um5']
matrix.fillna(0, inplace=True)
# # 修改age_range字段名称为 age_0, age_1, age_2... age_8
temp = pd.get_dummies(matrix['age_range'], prefix='age')
matrix = pd.concat([matrix, temp], axis=1)
temp = pd.get_dummies(matrix['gender'], prefix='g')
matrix = pd.concat([matrix, temp], axis=1)
matrix.drop(['age_range', 'gender'], axis=1, inplace=True)
matrix

,user_id,merchant_id,label,origin,u1,u2,u3,u4,u5,u6,...,age_2,age_3,age_4,age_5,age_6,age_7,age_8,g_0,g_1,g_2
0,365952,1203,0.0,train,46,29,12,16,16,4.933333,...,0,0,0,0,0,0,0,0,1,0
1,42624,946,0.0,train,365,198,46,46,45,5.866667,...,1,0,0,0,0,0,0,1,0,0
2,240000,2278,0.0,train,47,31,14,15,17,5.833333,...,0,1,0,0,0,0,0,1,0,0
3,177792,951,0.0,train,234,105,23,35,36,5.833333,...,0,0,0,0,0,0,0,0,1,0
4,322944,1892,0.0,train,186,106,34,40,39,5.866667,...,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23888,47231,1748,nan,test,128,97,28,39,40,5.816667,...,0,0,0,0,0,0,0,1,0,0
23889,59519,798,nan,test,1286,540,55,93,96,6.000000,...,0,1,0,0,0,0,0,1,0,0
23890,263039,639,nan,test,9,8,7,7,7,5.783333,...,1,0,0,0,0,0,0,0,1,0
23891,263039,3954,nan,test,9,8,7,7,7,5.783333,...,1,0,0,0,0,0,0,0,1,0


### 分割训练数据和测试数据

In [32]:
# 分割训练数据和测试数据
train_data = matrix[matrix['origin'] == 'train'].drop(['origin'], axis=1)
test_data = matrix[matrix['origin'] == 'test'].drop(['label', 'origin'], axis=1)
x, y = train_data.drop(['label'], axis=1), train_data['label']

In [33]:
x

,user_id,merchant_id,u1,u2,u3,u4,u5,u6,u7,u8,...,age_2,age_3,age_4,age_5,age_6,age_7,age_8,g_0,g_1,g_2
0,365952,1203,46,29,12,16,16,4.933333,45.0,0.0,...,0,0,0,0,0,0,0,0,1,0
1,42624,946,365,198,46,46,45,5.866667,313.0,0.0,...,1,0,0,0,0,0,0,1,0,0
2,240000,2278,47,31,14,15,17,5.833333,42.0,0.0,...,0,1,0,0,0,0,0,1,0,0
3,177792,951,234,105,23,35,36,5.833333,177.0,0.0,...,0,0,0,0,0,0,0,0,1,0
4,322944,1892,186,106,34,40,39,5.866667,147.0,0.0,...,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17832,411263,3826,40,24,13,14,15,4.916667,27.0,0.0,...,0,0,0,0,1,0,0,0,1,0
17833,415103,3444,15,12,10,10,10,6.000000,10.0,0.0,...,0,1,0,0,0,0,0,1,0,0
17834,90239,4173,16,14,8,11,12,5.833333,15.0,0.0,...,1,0,0,0,0,0,0,1,0,0
17835,355199,993,33,25,12,15,15,4.916667,31.0,0.0,...,0,0,0,0,1,0,0,1,0,0


In [34]:
y

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
17832    0.0
17833    0.0
17834    0.0
17835    0.0
17836    0.0
Name: label, Length: 17837, dtype: object

In [35]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x, y)
print(train_x.shape)
print(len(test_x))
print(train_y.shape)
print(len(test_y))

(13377, 45)
4460
(13377,)
4460


In [36]:
#导入模型 进行评估
from sklearn.tree import DecisionTreeClassifier
model_DT=DecisionTreeClassifier()
model_DT.fit(train_x,train_y)
print('模型的分数：',model_DT.score(test_x, test_y))

模型的分数： 0.8778026905829597


In [37]:
from sklearn.ensemble import RandomForestClassifier
model_RF=RandomForestClassifier(n_estimators=20,max_depth=15,random_state=50)
model_RF.fit(train_x,train_y)
print('模型的分数：',model_RF.score(test_x, test_y))

模型的分数： 0.9369955156950672


In [38]:
from sklearn.linear_model import LogisticRegression
model_LR=LogisticRegression()
model_LR.fit(train_x,train_y)
print('模型的分数：',model_LR.score(test_x, test_y))

模型的分数： 0.9367713004484305


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [39]:
from sklearn.tree import DecisionTreeClassifier
model_DT_e=DecisionTreeClassifier(criterion="entropy")
model_DT_e.fit(train_x,train_y)
print('模型的分数：',model_DT_e.score(test_x, test_y))

模型的分数： 0.8760089686098654


In [40]:
import xgboost as xgb
# 使用XGBoost
model_XGB = xgb.XGBClassifier(
    max_depth=7,
    n_estimators=1000,
    min_child_weight=300, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=0.3,    
    seed=42    
)
model_XGB.fit(
    train_x, train_y,
    eval_metric='auc', eval_set=[(train_x, train_y), (test_x, test_y)],
    verbose=True,
    #早停法，如果auc在10epoch没有进步就stop
    early_stopping_rounds=20 
)
print('模型的分数：',model_XGB.score(test_x, test_y))

[0]	validation_0-auc:0.58217	validation_1-auc:0.58558
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 20 rounds.
[1]	validation_0-auc:0.61610	validation_1-auc:0.62416
[2]	validation_0-auc:0.62000	validation_1-auc:0.62314
[3]	validation_0-auc:0.62693	validation_1-auc:0.63036
[4]	validation_0-auc:0.62719	validation_1-auc:0.63479
[5]	validation_0-auc:0.62701	validation_1-auc:0.62973
[6]	validation_0-auc:0.62317	validation_1-auc:0.62991
[7]	validation_0-auc:0.62355	validation_1-auc:0.63231
[8]	validation_0-auc:0.62495	validation_1-auc:0.64029
[9]	validation_0-auc:0.62196	validation_1-auc:0.63736
[10]	validation_0-auc:0.62066	validation_1-auc:0.63585
[11]	validation_0-auc:0.62026	validation_1-auc:0.63679
[12]	validation_0-auc:0.61979	validation_1-auc:0.63672
[13]	validation_0-auc:0.61858	validation_1-auc:0.63447
[14]	validation_0-auc:0.61801	validation_1-auc:0.63413
[15]	validation_0-auc:0.6176

In [41]:
#目前跑的这个分数 看来  是随机森林最好  然后就用随机森林看看分数吧
prob = model_RF.predict_proba(test_data)
submission['prob'] = pd.Series(prob[:,1])
submission.drop(['origin'], axis=1, inplace=True)
submission.to_csv('predictionRF.csv', index=False)
#提交之后的分数不是很理想  后续有时间再试试别的模型吧!